# Bayesian Time Series with hierarchical models

In [ ]:
import pymc as pm
import numpy as np
import arviz as az
import matplotlib.pyplot as plt


### 1. Generate Synthetic Time Series Data


In [5]:


np.random.seed(42)

# Global trend parameters
true_alpha_global = 5
true_beta_global = 0.5

# Number of "groups" or segments
num_groups = 3
group_lengths = [30, 40, 35] # Length of time series for each group

data = []
group_indices = []
time_points = []

# Generate data for each group
for i in range(num_groups):
    group_alpha_offset = np.random.normal(loc=0, scale=2) # Group-specific alpha offset
    group_beta_offset = np.random.normal(loc=0, scale=0.1) # Group-specific beta offset

    t = np.arange(group_lengths[i])
    y_true_group = (true_alpha_global + group_alpha_offset) + \
                   (true_beta_global + group_beta_offset) * t
    y_observed = y_true_group + np.random.normal(loc=0, scale=1.5, size=group_lengths[i])

    data.extend(y_observed)
    group_indices.extend([i] * group_lengths[i])
    time_points.extend(t)

data = np.array(data)
group_indices = np.array(group_indices)
time_points = np.array(time_points)

# Normalize time points for better numerical stability in regression
time_points_norm = (time_points - np.mean(time_points)) / np.std(time_points)

### 2. Define a Bayesian Hierarchical Model

In [6]:

with pm.Model() as hierarchical_time_series_model:
    # Global priors for the intercept and slope
    alpha_global = pm.Normal('alpha_global', mu=0, sigma=10)
    beta_global = pm.Normal('beta_global', mu=0, sigma=5)

    # Hierarchical priors for group-specific offsets
    # These offsets are drawn from a distribution whose parameters are also learned
    alpha_offset_sd = pm.HalfNormal('alpha_offset_sd', sigma=5)
    beta_offset_sd = pm.HalfNormal('beta_offset_sd', sigma=1)

    alpha_offset = pm.Normal('alpha_offset', mu=0, sigma=alpha_offset_sd, shape=num_groups)
    beta_offset = pm.Normal('beta_offset', mu=0, sigma=beta_offset_sd, shape=num_groups)

    # Group-specific intercepts and slopes
    alpha_group = pm.Deterministic('alpha_group', alpha_global + alpha_offset[group_indices])
    beta_group = pm.Deterministic('beta_group', beta_global + beta_offset[group_indices])

    # Expected value (mean) of the observations
    mu = alpha_group + beta_group * time_points_norm

    # Noise parameter for the observations
    sigma = pm.HalfNormal('sigma', sigma=2)

    # Likelihood (observed data)
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=data)

In [8]:
### 3. Infer Posterior Distributions
print("Sampling the posterior distribution...")
with hierarchical_time_series_model:
    idata = pm.sample(draws=2000, tune=2000, chains=2, cores=1, return_inferencedata=True)
    # cores=1 is used for reproducibility and simplicity; in production, use more cores.
print("Sampling complete.")

# 4. Analyze results
az.summary(idata, var_names=['alpha_global', 'beta_global', 'alpha_offset_sd', 'beta_offset_sd', 'sigma'])
print("\nGroup-specific parameters:")
az.summary(idata, var_names=['alpha_group', 'beta_group'], filter_vars="like", group="posterior")


# 5. Forecast (simple example: extend existing groups)
print("\nPerforming a simple forecast...")

# Forecast for an additional 10 time points for each group
forecast_horizon = 10
new_time_points_raw = []
new_group_indices = []

for i in range(num_groups):
    last_t_in_group = np.max(time_points[group_indices == i])
    new_t = np.arange(last_t_in_group + 1, last_t_in_group + 1 + forecast_horizon)
    new_time_points_raw.extend(new_t)
    new_group_indices.extend([i] * forecast_horizon)

new_time_points_raw = np.array(new_time_points_raw)
new_group_indices = np.array(new_group_indices)

# Normalize new time points using the *original* normalization parameters
new_time_points_norm = (new_time_points_raw - np.mean(time_points)) / np.std(time_points)

# Generate posterior predictive samples for forecasting
with hierarchical_time_series_model:
    pm.set_data({
        'time_points_norm': new_time_points_norm,
        'group_indices': new_group_indices
    })
    # Use 'y_obs' name for the predicted variable if it was used as observed.
    # Otherwise, define a new deterministic variable for prediction.
    # For prediction, we are interested in the 'mu' (mean)
    # and potentially the full predictive distribution if we include 'sigma' in the prediction.

    # Recreate the mu calculation for new data
    alpha_offset_pred = idata.posterior["alpha_offset"].values[:, :, new_group_indices].reshape(-1, len(new_group_indices))
    beta_offset_pred = idata.posterior["beta_offset"].values[:, :, new_group_indices].reshape(-1, len(new_group_indices))

    alpha_global_pred = idata.posterior["alpha_global"].values.flatten()
    beta_global_pred = idata.posterior["beta_global"].values.flatten()

    # Re-calculate group-specific parameters for each sample
    alpha_group_pred_samples = alpha_global_pred[:, np.newaxis] + alpha_offset_pred
    beta_group_pred_samples = beta_global_pred[:, np.newaxis] + beta_offset_pred

    mu_forecast_samples = alpha_group_pred_samples + beta_group_pred_samples * new_time_points_norm

    # If you want to sample from the predictive distribution (including observation noise)
    sigma_samples = idata.posterior["sigma"].values.flatten()
    y_forecast_samples = mu_forecast_samples + sigma_samples[:, np.newaxis] * np.random.randn(*mu_forecast_samples.shape)

# Calculate mean and credible interval for forecasts
forecast_mean = np.mean(y_forecast_samples, axis=0)
forecast_hdi = az.hdi(y_forecast_samples, hdi_prob=0.94) # 94% Highest Density Interval

# Plotting the results
plt.figure(figsize=(12, 7))

# Plot historical data
for i in range(num_groups):
    plt.scatter(time_points[group_indices == i], data[group_indices == i],
                label=f'Group {i} Historical Data', alpha=0.7)

# Plot forecast
for i in range(num_groups):
    group_forecast_mask = (new_group_indices == i)
    plt.plot(new_time_points_raw[group_forecast_mask], forecast_mean[group_forecast_mask],
             linestyle='--', color=f'C{i}', label=f'Group {i} Forecast Mean')
    plt.fill_between(new_time_points_raw[group_forecast_mask],
                     forecast_hdi[:, 0][group_forecast_mask],
                     forecast_hdi[:, 1][group_forecast_mask],
                     color=f'C{i}', alpha=0.2, label=f'Group {i} 94% HDI')

plt.title('Bayesian Hierarchical Time Series Forecasting')
plt.xlabel('Time Point')
plt.ylabel('Observed Value')
plt.legend()
plt.grid(True)
plt.show()


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1909, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    return self.fn(fgraph, node)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/tensor/rewriting/basic.py", line 1122, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/lin

Sampling the posterior distribution...

You can find the C code in this temporary file: /var/folders/rg/y2bhn3sn6yd6p5_pjx6snylm0000gn/T/pytensor_compilation_error_d8fvbzjz

You can find the C code in this temporary file: /var/folders/rg/y2bhn3sn6yd6p5_pjx6snylm0000gn/T/pytensor_compilation_error_3fe4mu85

You can find the C code in this temporary file: /var/folders/rg/y2bhn3sn6yd6p5_pjx6snylm0000gn/T/pytensor_compilation_error_0_t3sgdf


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1909, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    return self.fn(fgraph, node)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/tensor/rewriting/basic.py", line 1122, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/lin


You can find the C code in this temporary file: /var/folders/rg/y2bhn3sn6yd6p5_pjx6snylm0000gn/T/pytensor_compilation_error_eu_oo34a

You can find the C code in this temporary file: /var/folders/rg/y2bhn3sn6yd6p5_pjx6snylm0000gn/T/pytensor_compilation_error_mj0_kqgx


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1909, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    return self.fn(fgraph, node)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/tensor/rewriting/basic.py", line 1122, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/lin

...

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float32}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1909, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    return self.fn(fgraph, node)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/tensor/rewriting/basic.py", line 1122, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/link/c/o


You can find the C code in this temporary file: /var/folders/rg/y2bhn3sn6yd6p5_pjx6snylm0000gn/T/pytensor_compilation_error_59j3ln7a

You can find the C code in this temporary file: /var/folders/rg/y2bhn3sn6yd6p5_pjx6snylm0000gn/T/pytensor_compilation_error_tyoff_8a

You can find the C code in this temporary file: /var/folders/rg/y2bhn3sn6yd6p5_pjx6snylm0000gn/T/pytensor_compilation_error_x7i4i52h

You can find the C code in this temporary file: /var/folders/rg/y2bhn3sn6yd6p5_pjx6snylm0000gn/T/pytensor_compilation_error_l7r6i22z


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float32}(10)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1909, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    return self.fn(fgraph, node)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/tensor/rewriting/basic.py", line 1122, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytensor/link/c/


You can find the C code in this temporary file: /var/folders/rg/y2bhn3sn6yd6p5_pjx6snylm0000gn/T/pytensor_compilation_error_6a1qvobh


KeyboardInterrupt: 